In [11]:
from pathlib import Path
import sys
import os

FILE = Path(Path.cwd()).resolve()

ROOT = FILE.parents[0]  # YOLO root directory
# ROOT = FILE

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative
print(ROOT)
import glob

from draw_bbox import xywh2xyxy, calculate_iou, calculate_iou

..


In [12]:
IMG_FORMATS = 'bmp', 'dng', 'jpeg', 'jpg', 'mpo', 'png', 'tif', 'tiff', 'webp', 'pfm'  # include image suffixes

img_root = '/mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/**/*'
files = []
files.extend(sorted(glob.glob(img_root, recursive=True)))  # glob
images = [x for x in files if x.split('.')[-1].lower() in IMG_FORMATS]


In [13]:
print(len(images))
print(images)

857
['/mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/100m/30c/Crop_0010_100m_30c_1_2_00001.jpg', '/mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/100m/30c/Crop_0010_100m_30c_1_2_00002.jpg', '/mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/100m/30c/Crop_0010_100m_30c_1_2_00003.jpg', '/mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/100m/30c/Crop_0010_100m_30c_1_2_00004.jpg', '/mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/100m/30c/Crop_0015_100m_30c_2_3_00024.jpg', '/mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/100m/30c/Crop_0015_100m_30c_2_3_00025.jpg', '/mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/100m/30c/Crop_0015_100m_30c_2_3_00026.jpg', '/mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/100m/30c/Crop_0015_100m_30c_2_3_00027.jpg', '/mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/100m/30c/Crop_0015_100m_30c_2_3_00028.jp

In [14]:
from pathlib import Path
img_idx = 101
p = Path(images[img_idx])
print(f'source: {images[img_idx]}')
print(f'parent: {p.parent}')
print(f'name: {p.name}')
print(f'suffix: {p.suffix}')
print(f'stem: {p.stem}')


source: /mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/10m/30c/Crop_0012_010m_30c_2_3_00115.jpg
parent: /mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/10m/30c
name: Crop_0012_010m_30c_2_3_00115.jpg
suffix: .jpg
stem: Crop_0012_010m_30c_2_3_00115


In [15]:
import numpy as np

label_file = p.parent / f'{p.stem}.txt'
print(label_file)

/mnt/hdd01/hyeongdo/datasets/VR-DRONE-v1.0.0.test/20221207/1Class/10m/30c/Crop_0012_010m_30c_2_3_00115.txt


In [16]:
import cv2
image = cv2.imread(images[img_idx])
w, h = image.shape[:2]

In [17]:
try:
    with open(label_file) as fd:
        label = np.array(list(map(lambda x:np.array(x.strip().split(), dtype=np.float32), fd.readlines())))
        label[:, 1:] = xywh2xyxy(label[:, 1:])
        label[:, [1, 3]] *= w
        label[:, [2, 4]] *= h
except:
    label = np.array([])

In [18]:
if label.size > 0:
    label[:1,]

In [19]:
pred_path = '/mnt/hdd01/hyeongdo/workspace/yolov9/runs/detect/motsynth_48e_1920/labels'
pred_file = os.path.join(pred_path, f'{p.stem}.txt')
with open(pred_file) as fd:
    pred = np.array(list(map(lambda x:np.array(x.strip().split(), dtype=np.float32), fd.readlines())))
    pred[:, 1:5] = xywh2xyxy(pred[:, 1:5])
    pred[:, [1, 3]] *= w
    pred[:, [2, 4]] *= h
    pred = list(pred)

In [20]:
print(type(label), type(pred))
print(label.shape)
# print(label[1])
print(pred.shape)
ious = calculate_iou(label[0:1, 1:], pred[:,1:])[0]
print(ious)
ious_argsort = ious.argsort() # array를 정렬하는 index array를 반환
print(ious_argsort)
# for i in range(label.shape[0]):
#     print(i)

<class 'numpy.ndarray'> <class 'list'>
(0,)


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
x11, y11, x12, y12 = np.split(label[:1, 1:], 4, axis=1)
x21, y21, x22, y22 = np.split(pred[:, 1:], 4, axis=1)

In [ ]:
print(x11)
print(x21)
print(pred[:,1:2])
print(np.transpose(x21))
print(np.maximum(x11, np.transpose(x21)))
print(np.maximum(x11, x21))

In [ ]:
a11 = np.array([[259.31232]])
a21 = np.array([[265.49997],[250.43732]])
print(a11)
print(a21)
print(np.maximum(a11, np.transpose(a21)))

In [ ]:
xa = np.maximum(x11, np.transpose(x21))
xb = np.maximum(x12, np.transpose(x22))
ya = np.maximum(y11, np.transpose(y21))
yb = np.maximum(y12, np.transpose(y22))

In [ ]:
print(x11, x21)
print(xa, xb, ya, yb)

In [ ]:
# for i in range(pred.shape[0]):
#     print(type(i))
print(label[0:1,].shape)
print(np.squeeze(label[:1,], axis=1))
print(label[:1,].shape)
ious = calculate_iou(pred[:, 1:], label[:1, 1:])